In [1]:
import numpy as np

from Configuration import Configuration
from Oracle import Oracle
#from Model import TBP_AS_model

filename = "../UD_French-GSD/UD_French-GSD/fr_gsd-ud-train.conllu"
#filename = "../UD_French-GSD/UD_French-GSD/test.conllu"

oracle = Oracle(filename)
oracle.search_transitions()

In [2]:
x = np.array(oracle.features)
y = np.array(oracle.transitions)

y = np.array([k['transition']for k in y])
#shift:0, right:1, left:2
new_y = []
for i in range(len(y)):
    if y[i]=='shift':
        new_y.append([1,0,0])
    elif y[i]=='right':
        new_y.append([0,1,0])
    elif y[i]=='left':
        new_y.append([0,0,1])
        
y = np.array(new_y)
x.shape, y.shape

Using TensorFlow backend.


((621719, 10), (621719, 3))

In [13]:
import time
t = time.time()
vocs, inverses = zip(*(np.unique(feature, return_inverse=True) for feature in x.T))
x_new = np.vstack(inverses).T
print(time.time()-t)

7.1967175006866455


In [3]:
vocs = [np.array(list(set(x[:,i]))) for i in range(np.shape(x)[1])]
x_new = [[np.argwhere(vocs[j]==x[i,j]) for j,feature in enumerate(features)] for i,features in enumerate(x)]

In [4]:
x_new = np.squeeze(x_new)

In [5]:
np.shape(x_new)

(621719, 10)

In [6]:
from keras.models import Sequential, Model
from keras.layers import Dense, Embedding, Flatten, Input, Dropout, BatchNormalization, Input, Concatenate
from keras.optimizers import Adam

def build_model(vocs):
    n_features = len(vocs)
    embedding_size_max = 32
    
    input_features = [Input(shape=(1,)) for _ in range(n_features)]
    features = [Embedding(len(vocs[i]), min(embedding_size_max, len(vocs[i])))(f) for i,f in enumerate(input_features)]
    features = [Flatten()(f) for i,f in enumerate(features)]
    features = Concatenate()(features)
    
    input_shape = np.sum([min(embedding_size_max, len(vocs[i])) for i,f in enumerate(input_features)])
    
    model = Sequential()
    model.add(Dense(input_shape, input_shape=(input_shape,), activation='relu'))
    model.add(Dense(128, activation='relu'))
    model.add(BatchNormalization())
    model.add(Dropout(0.4))
    model.add(Dense(128, activation='relu'))
    model.add(BatchNormalization())
    model.add(Dropout(0.4))
    model.add(Dense(64, activation='relu'))
    model.add(BatchNormalization())
    model.add(Dropout(0.4))
    model.add(Dense(32, activation='relu'))
    model.add(BatchNormalization())
    model.add(Dropout(0.4))
    model.add(Dense(32, activation='relu'))
    model.add(BatchNormalization())
    model.add(Dropout(0.4))
    model.add(Dense(16, activation='relu'))
    model.add(BatchNormalization())
    model.add(Dropout(0.4))
    model.add(Dense(8, activation='relu'))
    model.add(BatchNormalization())

    output = model(features)

    transition = Dense(3, activation="tanh")(output)

    classifier =  Model(input_features, transition)
    optimizer = Adam(lr=0.0001)
    losses = ['categorical_crossentropy']
    classifier.compile(loss=losses,
                optimizer='adam',
                metrics=['accuracy'])

    classifier.summary()
    return classifier

classifier = build_model(vocs=vocs)

__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_1 (InputLayer)            (None, 1)            0                                            
__________________________________________________________________________________________________
input_2 (InputLayer)            (None, 1)            0                                            
__________________________________________________________________________________________________
input_3 (InputLayer)            (None, 1)            0                                            
__________________________________________________________________________________________________
input_4 (InputLayer)            (None, 1)            0                                            
__________________________________________________________________________________________________
input_5 (I

In [6]:
from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test = train_test_split(x_new, y, test_size=0.2, random_state=42)

In [7]:
X_train[:10].T.tolist()

[[4, 4, 5, 5, 12, 11, 1, 12, 4, 10],
 [5310, 5310, 3431, 11593, 15039, 2857, 1832, 1654, 9903, 12901],
 [0, 0, 108, 5, 113, 65, 0, 42, 0, 34],
 [14, 14, 5, 12, 14, 10, 18, 16, 18, 7],
 [9, 13, 11, 5, 4, 5, 5, 12, 10, 11],
 [21424, 26933, 11408, 5891, 11227, 5327, 25398, 22956, 24749, 26067],
 [0, 0, 46, 7, 0, 128, 97, 7, 104, 46],
 [1, 16, 16, 4, 5, 9, 8, 11, 12, 4],
 [12, 9, 9, 9, 9, 9, 12, 5, 9, 10],
 [29, 63, 108, 33, 108, 94, 29, 63, 93, 29]]

In [9]:
import matplotlib.pyplot as plt
def plot_hist(hist):
    
    plt.figure(figsize=(15,5))
    
    plt.subplot(1,2,1)
    plt.title('Loss')
    plt.plot(hist.history['loss'], label='train')
    plt.plot(hist.history['val_loss'], label='valid')
    plt.yscale('log')
    plt.ylabel('categorical_crossentropy')
    plt.xlabel('epoch')
    plt.grid(which='both')
    plt.legend()
    
    plt.subplot(1,2,2)
    plt.title('Accuracy')
    plt.plot(hist.history['acc'], label='train')
    plt.plot(hist.history['val_acc'], label='valid')
    plt.yscale('log')
    plt.ylabel('accuracy')
    plt.xlabel('epoch')
    plt.grid(which='both')
    plt.legend()
    
    plt.show()

In [12]:
def train_classif(classifier, X_train, y_train, X_test, y_test, batch_size=1024, epochs=10):
    hist = classifier.fit(
        X_train.T.tolist(),
        y_train,
        batch_size=batch_size,
        epochs=epochs,
        verbose=1,
        validation_data=(X_test.T.tolist(), y_test)
    )
    plot_hist(hist)

In [8]:
from sklearn.model_selection import cross_val_score
from sklearn.tree import DecisionTreeClassifier
clf = DecisionTreeClassifier(random_state=0)

#clf.fit(X_train, y_train)
#clf.score(X_test, y_test)

res = cross_val_score(clf, X_train, y_train, cv=10, n_jobs=8)

In [9]:
res

array([0.88535928, 0.8850376 , 0.88700792, 0.88622381, 0.88423338,
       0.88561835, 0.88768925, 0.88175805, 0.88811147, 0.88535698])

In [10]:
clf.fit(X_train, y_train)
clf.score(X_test, y_test)

0.8868944219262691

In [24]:
X_train.shape, y_train.shape

((464151, 10), (464151, 3))

In [11]:
train_classif(classifier, X_train, y_train, X_test, y_test, batch_size=1024, epochs=10)

Train on 464151 samples, validate on 228612 samples
Epoch 1/10
464151/464151 [==============================] - 20s 43us/step - loss: 3.2132 - acc: 0.3263 - val_loss: 1.3624 - val_acc: 0.2256
Epoch 2/10
464151/464151 [==============================] - 12s 26us/step - loss: 2.2454 - acc: 0.3012 - val_loss: 1.4883 - val_acc: 0.3105
Epoch 3/10
464151/464151 [==============================] - 11s 24us/step - loss: 1.9352 - acc: 0.3005 - val_loss: 1.2873 - val_acc: 0.2593
Epoch 4/10
464151/464151 [==============================] - 12s 25us/step - loss: 1.7622 - acc: 0.3043 - val_loss: 1.2710 - val_acc: 0.3068
Epoch 5/10
464151/464151 [==============================] - 14s 30us/step - loss: 1.5668 - acc: 0.3142 - val_loss: 1.2400 - val_acc: 0.2721
Epoch 6/10
464151/464151 [==============================] - 10s 23us/step - loss: 1.5165 - acc: 0.3045 - val_loss: 1.1001 - val_acc: 0.2358 - loss: 1.5092 - - ET - ETA: 4s - loss: 1.5204 - acc: 0.30 - ETA: 1s - loss: 1.5166 - a - ETA: 1s - loss: 1


NameError: name 'hist1' is not defined

In [ ]:
train_classif(classifier, X_train, y_train, X_test, y_test, batch_size=1024, epochs=10)

In [ ]:
train_classif(classifier, X_train, y_train, X_test, y_test, batch_size=512, epochs=10)

In [ ]:
train_classif(classifier, X_train, y_train, X_test, y_test, batch_size=256, epochs=10)

In [ ]:
train_classif(classifier, X_train, y_train, X_test, y_test, batch_size=128, epochs=10)

In [ ]:
train_classif(classifier, X_train, y_train, X_test, y_test, batch_size=64, epochs=10)

In [ ]:
hist4 = classifier.fit(X_train.T.tolist(), y_train, batch_size=512, epochs=10, verbose=1, validation_data=(X_test.T.tolist(), y_test), shuffle=True)

In [ ]:
plot_hist(hist4)